In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('data_sh/sentecnes_600.csv',names=['question', 'type'], header=None).sample(frac=1).reset_index(drop=True)

In [10]:
df.head()

,question,type
0,How to specify the impact energy and impact te...,Applying
1,Which of the following is the method for deter...,Remembering
2,Express the following systems in matrix form,Understanding
3,How do you construct foliations on a given man...,Applying
4,Which technology is used by laser printer,Remembering


In [14]:
from sklearn.preprocessing import OneHotEncoder

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['type'])

In [18]:
df['int_type'] = integer_encoded

In [22]:
import numpy as np

In [27]:
df_test = pd.read_csv('data_sh/test.csv', names=['question', 'type'], header=None)

In [28]:
df_test.head()

,question,type
0,List two reference parameters in the setHour f...,Remembering
1,Explain briefly the meaning of the following t...,Remembering
2,Label the parts of the diagram,Remembering
3,"Based on the above dataType class, list all th...",Remembering
4,Define morphology,Remembering


In [29]:
int_test = label_encoder.fit_transform(df_test['type'])

In [30]:
df_test['int_type'] = int_test

In [194]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clf', MultinomialNB()),])

In [38]:
text_clf = text_clf.fit(df.question, df.int_type)

In [39]:
predicted = text_clf.predict(df_test.question)
np.mean(predicted == df_test.int_type)

0.6879432624113475

In [141]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [202]:
# use this
text_clf_svm = Pipeline([
                         ('tfidf', TfidfVectorizer()),
                         ('clf-svm', LinearSVC(C=.4))
                        ])
text_clf_svm = text_clf_svm.fit(df.question, df.int_type)
predicted_svm = text_clf_svm.predict(df_test.question)
np.mean(predicted_svm == df_test.int_type)

0.7163120567375887

In [164]:
from sklearn.model_selection import GridSearchCV

In [165]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf-svm__C': (100, 1e-3),
             }

In [166]:
gs_clf = GridSearchCV(text_clf_svm, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(df.question, df.int_type)

/home/sourabh/ml/env/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [171]:
gs_clf.best_score_, gs_clf.best_params_
# predicted_dtf = gs_clf.predict(df_test.question)
# np.mean(predicted_dt == df_test.int_type)

(0.7516666666666667,
 {'clf-svm__C': 100, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)})

In [118]:
from sklearn.tree import DecisionTreeClassifier
text_clf_dt = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))),
                         ('tfidf', TfidfTransformer()),
                         ('clf-dt', DecisionTreeClassifier(random_state=49)),
                        ])

In [114]:
parameters = {'clf-dt__max_leaf_nodes': list(range(2, 100)), 'clf-dt__min_samples_split': [2, 3, 4, 5, 6]}

In [115]:
gs_clf_dt = GridSearchCV(text_clf_dt, parameters, n_jobs=-1)
gs_clf_dt = gs_clf_dt.fit(df.question, df.int_type)

In [116]:
gs_clf_dt.best_score_, gs_clf_dt.best_params_

(0.51, {'clf-dt__max_leaf_nodes': 98, 'clf-dt__min_samples_split': 3})

In [119]:
text_clf_dt = text_clf_dt.fit(df.question, df.int_type)

In [188]:
predicted_dt = text_clf_dt.predict(df_test.question)
np.mean(predicted_dt == df_test.int_type)

0.46808510638297873

# Final Classifier

In [217]:
text_clf_svm = Pipeline([
                         ('tfidf', TfidfVectorizer()),
                         ('clf-svm', LinearSVC(C=.4))
                        ])
text_clf_svm = text_clf_svm.fit(df.question, df.int_type)
predicted_svm = text_clf_svm.predict(df_test.question)
np.mean(predicted_svm == df_test.int_type)

0.7163120567375887

In [218]:
from sklearn.metrics import confusion_matrix, classification_report

In [220]:
print(confusion_matrix(df_test.int_type, predicted_svm))

[[14  2  1  3  2  1]
 [ 0  5  6  0  4  0]
 [ 0  4 24  0  0  2]
 [ 0  1  2 19  1  1]
 [ 0  2  0  0 23  1]
 [ 1  0  1  0  5 16]]


In [221]:
print(classification_report(df_test.int_type, predicted_svm))

              precision    recall  f1-score   support

           0       0.93      0.61      0.74        23
           1       0.36      0.33      0.34        15
           2       0.71      0.80      0.75        30
           3       0.86      0.79      0.83        24
           4       0.66      0.88      0.75        26
           5       0.76      0.70      0.73        23

    accuracy                           0.72       141
   macro avg       0.71      0.69      0.69       141
weighted avg       0.73      0.72      0.71       141



In [207]:
df_tr_mod = df[df['int_type'] != 1]

In [208]:
df_test_mod = df_test[df_test['int_type']!=1]